https://cssanalytics.wordpress.com/2020/12/23/how-should-trend-followers-adjust-to-the-modern-environment-enter-adaptive-momentum/

In [ ]:
from datetime import datetime

import pandas as pd
import numpy as np
import yfinance as yf
import plotly.express as px

def sharpe_ratio(ret, N=252, verbose=True):
    ret = ret[ret!=0]
    sr = np.mean(ret) / np.std(ret) * np.sqrt(N)
    if verbose:
        print(f'SR (rf=0): {sr}')
    return sr

def compound_annual_growth_rate(cpnl, verbose=True):
    t = ((cpnl.index[-1] - cpnl.index[0]) / 365.25)
    t = (t.seconds / (24 * 60 * 60) + t.days)
    FV = cpnl.values[-1] / cpnl.values[0]
    cagr = FV ** (1/t) - 1
    if verbose:
        print(f'CAGR: {cagr}')
    return cagr


def annual_returns(df, col='ret'):
    return df.groupby(pd.Grouper(freq='Y'), group_keys=False)[col].agg(lambda x: (1+x).prod() - 1).to_frame()


def dd_curve(df):
    df['dd'] = dd_curve_numba(df.cpnl.values, df.ret.values)
    return df

from numba import njit
@njit
def dd_curve_numba(cpnl, ret):
    N = len(cpnl)
    i=1
    hw = cpnl[1]
    dd = np.ones(N)
    while i < N:
        if hw > cpnl[i]:
            dd[i] = dd[i-1] * (1 + ret[i])
        else:
            dd[i] = 1
            hw = cpnl[i]
        i+=1
    return dd - 1
    

def performance_stats(df, N=12, verbose = True):
    sr = sharpe_ratio(df.ret, N, verbose)
    cagr = compound_annual_growth_rate(df.cpnl, verbose)
    df = dd_curve(df)
    max_dd = np.abs(df.dd.min())
    if verbose:
        print(f'Max draw down {max_dd}')
    pm = pd.DataFrame({'sr': sr, 'cagr': cagr, 'max_dd': max_dd}, index=[0])
    return df, pm


In [ ]:
# offensive_universe = ['SPY', 'IWM', 'EFA', 'EEM', 'VNQ', 'DBC', 'TLT']

US_Equities = ['SPY', 'IWM', 'QQQ', 'SPXL', 'TQQQ', 'SPY.AX', '']
Foreign_Equities = ['VEA', 'VWO']
Alternative_Assets = ['DBC', 'VNQ', 'GLD']
US_Bonds = ['IEF', 'TLT']


# SPY, IWM, VEA, VWO, VNQ, DBC, IEF, TLT 

offensive_universe = US_Equities + Foreign_Equities + Alternative_Assets + US_Bonds

cash_universe = ['BIL', 'IEF']
canary_asset = ['TIP']

# #### leveraged assets
# offensive_universe = ['SSO', 'UWM', 'VEA', 'VWO', 'DBC', 'URE', 'UBT', 'UST']

# cash_universe = ['BIL', 'UST']
# canary_asset = ['TIP']


whole_universe = offensive_universe + cash_universe + canary_asset
whole_universe = list(set(whole_universe))

In [ ]:
prices_df = yf.download(tickers=whole_universe,
                        end=datetime.now())


In [ ]:
prices_df = prices_df['Adj Close']

In [ ]:
fig = px.line(prices_df.dropna().melt(ignore_index=False), y='value', color='variable', log_y=True) 
fig.show()
# py.plot(fig, filename='HAA_universe_price_history')

In [ ]:
from numba import njit
@njit
def ecdf_dd_curve_numba(cpnl, ret):
    N = len(cpnl)
    i=1
    hw = cpnl[1]
    dd = np.ones(N)
    ecdf = np.zeros(N)
    while i < N:
        if hw > cpnl[i]:
            dd[i] = dd[i-1] * (1 + ret[i])
        else:
            dd[i] = 1
            hw = cpnl[i]

        ecdf[i] = np.sum(dd[i] < dd[:i]) / i

        i+=1
    return dd - 1, ecdf

In [ ]:
asset = 'SPY.AX'

def aema(df, asset, s_win = 250, f_win = 50):
    sma_slow = df[asset].ewm(span=s_win).mean()
    sma_fast = df[asset].ewm(span=f_win).mean()
    df['ama'] = (sma_slow * (1 - df.ecdf)) + (sma_fast * df.ecdf)
    return df

def ama_performance(df, asset, f_win, verbose=False):
    df['ret'] = df.asset_ret.shift(-1).fillna(0)
    mask = df[asset].ewm(f_win).mean() <= df.ama
    # mask = df[asset] <= df.ama
    # mask = df[asset] <= df.ama
    df.loc[mask, 'ret'] = 0
    df['cpnl'] = (1 + df.ret).cumprod()
    df, pm = performance_stats(df, N=252, verbose=verbose)
    return df, pm

pms = []

for f, p in product(np.arange(0, 55, 5)[1:], np.arange(0, 55, 5)[1:]):
    try:
        asset_ret_df = prices_df[[asset]].dropna().copy()
        asset_ret_df[asset] /= asset_ret_df[asset].values[0]
        asset_ret_df['asset_ret'] = asset_ret_df[asset].pct_change().fillna(0)

        _ = ecdf_dd_curve_numba(asset_ret_df[asset].values, asset_ret_df.asset_ret.values)
        asset_ret_df['dd'] = _[0]
        asset_ret_df['ecdf'] = _[1]
        asset_ret_df = aema(asset_ret_df, asset, 200, f)
        df, pm = ama_performance(asset_ret_df, asset, p, verbose=False)
        pm['price'] = p
        pm['fast'] = f
        pms.append(pm)
    except Exception as e:
        print(repr(e))
        # print('failed')
pms = pd.concat(pms).reset_index(drop=True)

print(pms.describe())
print(pms.iloc[pms.sr.idxmax()])
print(pms.iloc[pms.cagr.idxmax()])
print(pms.iloc[pms.max_dd.idxmin()])

best_pm = pms.iloc[pms.sr.idxmax()]

asset_ret_df = prices_df[[asset]].dropna().copy()
asset_ret_df[asset] /= asset_ret_df[asset].values[0]
asset_ret_df['asset_ret'] = asset_ret_df[asset].pct_change().fillna(0)

_ = ecdf_dd_curve_numba(asset_ret_df[asset].values, asset_ret_df.asset_ret.values)
asset_ret_df['dd'] = _[0]
asset_ret_df['ecdf'] = _[1]
asset_ret_df = aema(asset_ret_df, asset, 200, best_pm.fast)
asset_ret_df, pm = ama_performance(asset_ret_df, asset, best_pm.price, verbose=True)


plot_df = asset_ret_df.melt(ignore_index=False, value_vars=[asset, 'ama', 'cpnl', 'dd', 'ecdf'])
plot_df['facet'] = ''
plot_df.loc[plot_df.variable.isin([asset, 'ama']), 'facet'] = 'price'
plot_df.loc[plot_df.variable.isin(['cpnl']), 'facet'] = 'cpnl'
plot_df.loc[plot_df.variable.isin(['dd', 'ecdf']), 'facet'] = 'signal'
plot_df.loc[plot_df.variable== 'dd', 'value'] *= 10

fig = px.line(plot_df, y='value', color='variable', facet_row='facet', log_y=True)#.show()
fig['layout']['yaxis']['type'] = None
fig.update_yaxes(matches=None)
fig.update_layout(height = 1000)

In [ ]:
asset = 'TLT'

def aema(df, asset, s_win = 250, f_win = 50):
    sma_slow = df[asset].ewm(span=s_win).mean()
    sma_fast = df[asset].ewm(span=f_win).mean()
    df['ama'] = (sma_slow * (1 - df.ecdf)) + (sma_fast * df.ecdf)
    return df

def ama_performance(df, asset, f_win, verbose=False):
    df['ret'] = df.asset_ret.shift(-1).fillna(0)
    df.loc[df[asset].ewm(f_win).mean() <= df.ama, 'ret'] = 0
    df['cpnl'] = (1 + df.ret).cumprod()
    df, pm = performance_stats(df, N=252, verbose=verbose)
    return df, pm

pms = []

for f, p in product(np.arange(0, 55, 5)[1:], np.arange(0, 55, 5)[1:]):
    try:
        asset_ret_df = prices_df[[asset]].dropna().copy()
        asset_ret_df[asset] /= asset_ret_df[asset].values[0]
        asset_ret_df['asset_ret'] = asset_ret_df[asset].pct_change().fillna(0)

        _ = ecdf_dd_curve_numba(asset_ret_df[asset].values, asset_ret_df.asset_ret.values)
        asset_ret_df['dd'] = _[0]
        asset_ret_df['ecdf'] = _[1]
        asset_ret_df = aema(asset_ret_df, asset, 200, f)
        df, pm = ama_performance(asset_ret_df, asset, p, verbose=False)
        pm['price'] = p
        pm['fast'] = f
        pms.append(pm)
    except Exception as e:
        print(repr(e))
        # print('failed')
pms = pd.concat(pms).reset_index(drop=True)

print(pms.describe())
print(pms.iloc[pms.sr.idxmax()])
print(pms.iloc[pms.cagr.idxmax()])
print(pms.iloc[pms.max_dd.idxmin()])

best_pm = pms.iloc[pms.sr.idxmax()]

asset_ret_df = prices_df[[asset]].dropna().copy()
asset_ret_df[asset] /= asset_ret_df[asset].values[0]
asset_ret_df['asset_ret'] = asset_ret_df[asset].pct_change().fillna(0)

_ = ecdf_dd_curve_numba(asset_ret_df[asset].values, asset_ret_df.asset_ret.values)
asset_ret_df['dd'] = _[0]
asset_ret_df['ecdf'] = _[1]
asset_ret_df = aema(asset_ret_df, asset, 200, best_pm.fast)
asset_ret_df, pm = ama_performance(asset_ret_df, asset, best_pm.price, verbose=True)


plot_df = asset_ret_df.melt(ignore_index=False, value_vars=[asset, 'ama', 'cpnl', 'dd', 'ecdf'])
plot_df['facet'] = ''
plot_df.loc[plot_df.variable.isin([asset, sma_fast, sma_slow, 'ama']), 'facet'] = 'price'
plot_df.loc[plot_df.variable.isin(['cpnl']), 'facet'] = 'cpnl'
plot_df.loc[plot_df.variable.isin(['dd', 'ecdf']), 'facet'] = 'signal'
plot_df.loc[plot_df.variable== 'dd', 'value'] *= 10

fig = px.line(plot_df, y='value', color='variable', facet_row='facet', log_y=True)#.show()
fig['layout']['yaxis']['type'] = None
fig.update_yaxes(matches=None)
fig.update_layout(height = 1000)

In [ ]:
asset = 'SPY.AX'

def aema(df, asset, f_win = 50, p_win=10):
    sma_slow = df[asset].ewm(span=200).mean()
    sma_fast = df[asset].ewm(span=f_win).mean()
    df['ama'] = (sma_slow * (1 - df.ecdf)) + (sma_fast * df.ecdf)

    sma_slow = df[asset].ewm(span=f_win).mean()
    sma_fast = df[asset].ewm(span=p_win).mean()
    df['pama'] = (sma_slow * (1 - df.ecdf)) + (sma_fast * df.ecdf)
    return df.pama <= df.ama

def ama_performance(df, mask, verbose=False):
    df['ret'] = df.asset_ret.shift(-1).fillna(0)
    df.loc[mask, 'ret'] = 0
    df['cpnl'] = (1 + df.ret).cumprod()
    df, pm = performance_stats(df, N=252, verbose=verbose)
    return df, pm

pms = []

for f, p in product(np.arange(0, 55, 5)[1:], np.arange(0, 55, 5)[1:]):
    try:
        asset_ret_df = prices_df[[asset]].dropna().copy()
        asset_ret_df[asset] /= asset_ret_df[asset].values[0]
        asset_ret_df['asset_ret'] = asset_ret_df[asset].pct_change().fillna(0)

        _ = ecdf_dd_curve_numba(asset_ret_df[asset].values, asset_ret_df.asset_ret.values)
        asset_ret_df['dd'] = _[0]
        asset_ret_df['ecdf'] = _[1]
        mask = aema(asset_ret_df, asset, f, p)
        df, pm = ama_performance(asset_ret_df, mask, verbose=False)
        pm['price'] = p
        pm['fast'] = f
        pms.append(pm)
    except Exception as e:
        print(repr(e))
        # print('failed')
pms = pd.concat(pms).reset_index(drop=True)

print(pms.describe())
print(pms.iloc[pms.sr.idxmax()])
print(pms.iloc[pms.cagr.idxmax()])
print(pms.iloc[pms.max_dd.idxmin()])

best_pm = pms.iloc[pms.sr.idxmax()]

asset_ret_df = prices_df[[asset]].dropna().copy()
asset_ret_df[asset] /= asset_ret_df[asset].values[0]
asset_ret_df['asset_ret'] = asset_ret_df[asset].pct_change().fillna(0)

_ = ecdf_dd_curve_numba(asset_ret_df[asset].values, asset_ret_df.asset_ret.values)
asset_ret_df['dd'] = _[0]
asset_ret_df['ecdf'] = _[1]
mask = aema(asset_ret_df, asset, best_pm.fast, best_pm.price)
asset_ret_df, pm = ama_performance(asset_ret_df, mask, verbose=True)


plot_df = asset_ret_df.melt(ignore_index=False, value_vars=[asset, 'ama', 'pama', 'cpnl', 'dd', 'ecdf'])
plot_df['facet'] = ''
plot_df.loc[plot_df.variable.isin([asset, 'ama', 'pama']), 'facet'] = 'price'
plot_df.loc[plot_df.variable.isin(['cpnl']), 'facet'] = 'cpnl'
plot_df.loc[plot_df.variable.isin(['dd', 'ecdf']), 'facet'] = 'signal'
plot_df.loc[plot_df.variable== 'dd', 'value'] *= 10

fig = px.line(plot_df, y='value', color='variable', facet_row='facet', log_y=True)#.show()
fig['layout']['yaxis']['type'] = None
fig.update_yaxes(matches=None)
fig.update_layout(height = 1000)

In [ ]:
plot_df = spy_ret_df.melt(ignore_index=False, value_vars=['SPY', sma_fast, sma_slow, 'ama', 'cpnl', 'dd', 'ecdf'])
plot_df['facet'] = ''
plot_df.loc[plot_df.variable.isin(['SPY', sma_fast, sma_slow, 'ama']), 'facet'] = 'price'
plot_df.loc[plot_df.variable.isin(['cpnl']), 'facet'] = 'cpnl'
plot_df.loc[plot_df.variable.isin(['dd', 'ecdf']), 'facet'] = 'signal'
plot_df.loc[plot_df.variable== 'dd', 'value'] *= 10

fig = px.line(plot_df, y='value', color='variable', facet_row='facet', log_y=True)#.show()
fig['layout']['yaxis']['type'] = None
fig.update_yaxes(matches=None)
fig.update_layout(height = 1000)

In [ ]:
spy_ret_df.tail(12)


# SPY	spy_ret	dd	ecdf	sma200	sma20	ama	long	ret	cpnl	ama_l1
# Date											
# 2023-06-22	436.510010	0.003610	-0.017580	0.565661	395.048899	427.365677	413.329225	True	-0.007560	23672.445617	410.744344
# 2023-06-23	433.209991	-0.007560	-0.021594	0.588450	395.292541	428.547746	414.861583	True	-0.004086	23575.725852	411.827976
# 2023-06-26	431.440002	-0.004086	-0.010867	0.596473	395.492798	429.463974	415.755684	True	0.010963	23834.193870	412.900954
# 2023-06-27	436.170013	0.010963	-0.010368	0.568312	395.703913	430.349197	415.393259	True	0.000504	23846.215677	413.329225
# 2023-06-28	436.390015	0.000504	-0.006468	0.566410	395.885528	431.237450	415.909204	True	0.003941	23940.202228	414.861583
# 2023-06-29	438.109985	0.003941	0.000000	0.555759	396.054244	432.327768	416.213570	True	0.011801	24222.713860	415.755684
# 2023-06-30	443.279999	0.011801	0.000000	0.522392	396.336725	433.478822	415.739458	True	0.001151	24250.582974	415.393259
# 2023-07-03	443.790009	0.001151	-0.001487	0.517755	396.614376	434.351506	416.152947	True	-0.001487	24214.517553	415.909204
# 2023-07-05	443.130005	-0.001487	-0.009306	0.523561	396.910767	435.232037	416.974285	True	-0.007831	24024.901854	416.213570
# 2023-07-06	439.660004	-0.007831	-0.011807	0.546594	397.204450	435.892741	418.351222	True	-0.002525	23964.245842	415.739458
# 2023-07-07	438.549988	-0.002525	-0.009306	0.552917	397.477813	436.571672	419.093457	True	0.002531	24024.901854	416.152947
# 2023-07-10	439.660004	0.002531	-0.009306	0.546451	397.778757	437.177579	419.308280	True	0.000000	24024.901854	416.974285

In [ ]:
px.line(spy_ret_df, y='cpnl', log_y=True)

In [ ]:
class MOMENTUM():
    def __init__(self) -> None:
        pass

    def fit(self, X, y=None):
        return X
    
    def transform(self, X, y=None):
        self.calc_monthly_close(X)
        ret1m = self.get_return(1)
        ret3m = self.get_return(3)
        ret6m = self.get_return(6)
        ret12m = self.get_return(12)
        mom = (ret1m + ret3m + ret6m + ret12m) / 4
        return mom.shift(1)
    
    def calc_monthly_close(self, X):
        # last_month_date = X.reset_index().groupby(pd.Grouper(key = 'Date', freq='M')).Date.apply(lambda x: x[:-1])
        # self.monthly_close = X.loc[(last_month_date)].resample('M').last()
        self.monthly_close = X.resample('M').last()
    
    def get_return(self, months):
        return self.monthly_close.pct_change(periods = months)


class MONTHLY_RETURNS():
    def __init__(self) -> None:
        pass

    def fit(self, X, y=None):
        return X
    
    def calc_monthly_close(self, X):
        self.monthly_close = X.resample('M').last()
        # self.monthly_close = X.resample('M').first().shift(-1)

    def transform(self, X):
        self.calc_monthly_close(X)
        return self.get_return(1)

    def get_return(self, months):
        return self.monthly_close.pct_change()#periods = months)
    

class MONTHLY_PICKS():
    def __init__(self, offensive_universe, cash_universe, canary_asset) -> None:
        self.offensive_universe = offensive_universe
        self.cash_universe = cash_universe
        self.canary_asset = canary_asset
        self.regime = None
        
    def get_picks(self, X):
        self.get_regime(X)
        if self.regime:
            assets = self.get_offensive_assets(X)
        else:
            assets = self.get_defensive_assets(X)
        return assets, self.regime
    
    def get_regime(self, X):
        if X[canary_asset].values[0] > 0:
            self.regime = True
        else:
            self.regime = False

    def get_defensive_assets(self, X):
        asset = X[self.cash_universe].melt()
        # asset = asset[asset.value > 0]
        asset = asset.sort_values('value').tail(1).variable.tolist()
        return asset

    def get_offensive_assets(self, X):
        assets = X[self.offensive_universe].melt()
        assets = assets[assets.value > 0]
        assets = assets.sort_values('value').tail(4).variable.tolist()
        Cash_fraction = 4 - len(assets)
        if Cash_fraction > 0:
            defensive_asset = self.get_defensive_assets(X)
            assets = assets + defensive_asset * Cash_fraction
        return assets
    

class POSITION_SIZES():
    def __init__(self) -> None:
        pass

    def get_lots(self, capital, assets, prices):
        #### assets is a dict of aggressive
        N = len(assets)
        asset_capital = capital / N
        return [int(asset_capital / prices[asset].values[0]) for asset in assets]

    def get_lots_date(self, capital, assets, prices, date):
        return self.get_lots(capital, assets, prices.loc[date])

In [ ]:
prices_df.resample('M').first().pct_change()

In [ ]:
whole_universe

In [ ]:
mom = MOMENTUM()
mom_df = pd.concat([mom.transform(prices_df[sym]).to_frame() for sym in whole_universe], axis=1)
mom_df.dropna().to_csv('haa_momentum.csv')

In [ ]:
#### this matches the table at the end of
# https://indexswingtrader.blogspot.com/2023/02/introducing-hybrid-asset-allocation-haa.html

mom_df.tail(12)[['SPY', 'IWM', 'VEA', 'VWO', 'DBC', 'VNQ', 'TLT', 'IEF', 'BIL', 'TIP']].round(4) * 100

In [ ]:

monthly_ret = MONTHLY_RETURNS()
ret_df = pd.concat([monthly_ret.transform(prices_df[sym]).to_frame() for sym in whole_universe], axis=1)

position_sizes = POSITION_SIZES()

In [ ]:
px.line(mom_df.melt(ignore_index=False), y='value', color='variable')

In [ ]:
prices_df

In [ ]:
monthly_picks = MONTHLY_PICKS(offensive_universe, cash_universe, canary_asset)
monthly_assets = mom_df.dropna().groupby('Date').apply(monthly_picks.get_picks)
monthly_assets.tail(12)

In [ ]:
date = '2023-05-31'
capital = 20*10e3

position_sizes.get_lots(capital, monthly_assets.loc[date][0], prices_df.tail(1))#, pd.to_datetime(date))

In [ ]:
daily_ret_df = prices_df.pct_change().dropna().shift(-1)
daily_ret_df['Date'] = daily_ret_df.index.date + pd.tseries.offsets.MonthEnd(0)
daily_ret_df

In [ ]:
prices_df

In [ ]:
months_ret = []
months_regime = []
months_ret_daily = []

for Date in monthly_assets.index.unique():
    assets = monthly_assets.loc[Date][0] #.tolist()
    months_ret.append([Date, ret_df.loc[Date, assets].mean()])
    months_ret_daily.append(daily_ret_df.loc[daily_ret_df.Date == Date, assets].mean(axis=1))
    months_regime.append([Date, monthly_assets.loc[Date][1]])

strat_returns = pd.DataFrame(months_ret, columns=['Date', 'ret']).sort_values('Date').set_index('Date')
strat_regime = pd.DataFrame(months_regime, columns=['Date', 'offensive_regime']).sort_values('Date').set_index('Date')
strat_returns = pd.concat([strat_regime, strat_returns],axis=1)

strat_returns['regime_group'] = strat_returns.offensive_regime.diff().bfill().cumsum()
strat_returns['cpnl'] = (strat_returns.ret + 1).cumprod()

strat_returns['spy'] = prices_df.SPY.resample('M').last()
strat_returns['spy'] /= strat_returns.spy.values[0]
strat_returns['spy_ret'] = strat_returns.spy.pct_change().fillna(0)
strat_returns.tail(12)

In [ ]:
prices_df.BIL.resample('M').last().pct_change().tail(12)

In [ ]:
(91.446999 - 91.027702) / 91.027702

In [ ]:
prices_df.BIL.tail(30)

In [ ]:
ret_df.tail(12)[['BIL']]

In [ ]:
Date in monthly_assets.index.unique()

In [ ]:
months_ret_daily = pd.concat(months_ret_daily).dropna()
months_ret_daily = months_ret_daily.to_frame('ret')
months_ret_daily['cpnl'] = (months_ret_daily.ret + 1).cumprod()
months_ret_daily, pm_daily = performance_stats(months_ret_daily, N=252)

In [ ]:
months_ret_daily

In [ ]:
px.bar(strat_returns, y='ret', color='offensive_regime').show()
fig = px.line(strat_returns, y='cpnl', color='offensive_regime', line_group='regime_group', log_y=True)#.show()
benchmark_trace = px.line(strat_returns, y='spy').data[0]
benchmark_trace['line']['color'] = 'black'
benchmark_trace['line']['dash'] = 'dash'
fig.add_trace(benchmark_trace)
fig.show()
# py.plot(fig, filename='HAA_vs_SPY')

px.bar(strat_returns.ret.rolling(12).std() * np.sqrt(12)).show()
px.bar(annual_returns(strat_returns, 'ret'), y='ret').show()
strat_returns, pm = performance_stats(strat_returns, N=12)
pm['stratergy'] = 'HAA'

px.line(strat_returns, y='dd').show()

_, pm_spy = performance_stats(strat_returns.drop(['cpnl', 'ret'], axis=1).rename({'spy': 'cpnl', 'spy_ret': 'ret'}, axis=1), N=12)

pm_spy['stratergy'] = 'SPY'

pd.concat([pm, pm_spy]).set_index('stratergy').to_csv('HAA_pm_usd.csv')



In [ ]:
strat_returns[strat_returns.index.year.isin([2020,2021,2022])].copy()

In [ ]:
_, pm_3yr = performance_stats(strat_returns[strat_returns.index.year.isin([2020,2021,2022])].copy(), N=12)

In [ ]:
strat_returns[strat_returns.index.year.isin([2022])]

In [ ]:
_, pm_1yr = performance_stats(strat_returns[strat_returns.index.year.isin([2022])].copy(), N=12)

In [ ]:
usd_aud_df = pd.read_csv('../trading_data/AUD_USD Historical Data.csv').set_index('Date')
usd_aud_df.index = pd.to_datetime(usd_aud_df.index, dayfirst=True)

In [ ]:
strat_returns_aud = strat_returns.copy()
strat_returns_aud = pd.merge(strat_returns_aud, left_index = True, right = usd_aud_df[['Price']], right_index=True, how='left')
# strat_returns_aud[['ret', 'cpnl', 'spy']] = 
strat_returns_aud['ret'] /= strat_returns_aud.Price.ffill()
strat_returns_aud['cpnl'] = (strat_returns_aud.ret + 1).cumprod()
# strat_returns_aud['cpnl'] /= strat_returns_aud.Price.ffill()
strat_returns_aud['spy'] /= strat_returns_aud.Price.ffill()
# strat_returns_aud

In [ ]:
px.bar(strat_returns_aud, y='ret', color='offensive_regime').show()
fig = px.line(strat_returns_aud, y='cpnl', color='offensive_regime', line_group='regime_group', log_y=True)#.show()
benchmark_trace = px.line(strat_returns_aud, y='spy').data[0]
benchmark_trace['line']['color'] = 'black'
benchmark_trace['line']['dash'] = 'dash'
fig.add_trace(benchmark_trace)
fig.show()
strat_returns_aud, *_ = performance_stats(strat_returns_aud, N=12)
px.line(strat_returns_aud, y='dd').show()



In [ ]:
py.plot(fig, filename='HAA_equity')

In [ ]:

px.bar(strat_returns_aud.ret.rolling(12).std() * np.sqrt(12)).show()
px.bar(annual_returns(strat_returns_aud, 'ret'), y='ret').show()
cagr(strat_returns_aud.cpnl.dropna())
sr(strat_returns_aud.ret.dropna(), N=12)

In [ ]:
# CAGR: 0.10724184105915247
# SR: 1.1479426565313477

In [ ]:
cagr(strat_returns_aud.spy.dropna())
sr(strat_returns_aud.spy.pct_change().dropna(), N=12)

# CAGR: 0.09743790454842394
# SR: 1.0914094303126585

# CAGR: 0.09680215314678176
# SR: 1.140642818036425

In [ ]:
vol_df = np.log(prices_df)#.apply(lambda x: np.diff(np.log(x)))
vol_df = vol_df.diff().dropna()
vol_df = (vol_df.rolling(252).std() * np.sqrt(252)).mean(axis=1).to_frame('vol')\
       + (vol_df.rolling(189).std() * np.sqrt(189)).mean(axis=1).to_frame('vol')\
       + (vol_df.rolling(126).std() * np.sqrt(126)).mean(axis=1).to_frame('vol')\
       + (vol_df.rolling(63).std() * np.sqrt(63)).mean(axis=1).to_frame('vol')
vol_df /= 4
px.bar(vol_df.resample('M').mean()).show()
vol_df = vol_df.resample('M').mean().shift(1)
                    
#                     .rolling(252).std() * np.sqrt(252)).mean(axis=1).values
# prices_df#.groupby()

In [ ]:
px.bar(strat_returns.ret.rolling(12).std() * np.sqrt(12))

In [ ]:
max_leverage = 1
vol_window_l = 12
vol_window_mm = 9
vol_window_m = 6
vol_window_s = 3

vol_target = 0.1 #25

strat_returns['rolling_vol'] = ((strat_returns.ret.rolling(vol_window_l).std() * np.sqrt(12)).shift(1)\
                                 + (strat_returns.ret.rolling(vol_window_mm).std() * np.sqrt(12)).shift(1)\
                                 + (strat_returns.ret.rolling(vol_window_m).std() * np.sqrt(12)).shift(1)\
                                 + (strat_returns.ret.rolling(vol_window_s).std() * np.sqrt(12)).shift(1)) / 4
strat_returns['leverage'] = (vol_target / strat_returns.rolling_vol).dropna()
# strat_returns['leverage'] = vol_target / vol_df
mask = strat_returns.leverage > 1
strat_returns.loc[mask, 'leverage'] = 1
strat_returns.loc[~strat_returns.offensive_regime, 'leverage'] = 1
strat_returns['ret_l'] = strat_returns.ret * strat_returns.leverage
strat_returns['cpnl_l'] = (strat_returns.ret_l + 1).cumprod()
px.bar(strat_returns, y='ret_l', color='offensive_regime').show()
fig = px.line(strat_returns, y='cpnl_l', color='offensive_regime', line_group='regime_group', log_y=True)#.show()
benchmark_trace = px.line(strat_returns, y='spy').data[0]
benchmark_trace['line']['color'] = 'black'
benchmark_trace['line']['dash'] = 'dash'
fig.add_trace(benchmark_trace)
fig.show()
px.bar(strat_returns, y='leverage', color='offensive_regime').show()
px.bar(strat_returns.ret_l.rolling(12).std() * np.sqrt(12)).show()
px.bar(annual_returns(strat_returns, 'ret_l'), y='ret_l').show()
cagr(strat_returns.cpnl_l.dropna())
sr(strat_returns.ret_l.dropna(), N=12)

In [ ]:
max_leverage = 1
vol_window_l = 12
vol_window_mm = 9
vol_window_m = 6
vol_window_s = 3

vol_target = 0.1 #25

strat_returns_aud['rolling_vol'] = ((strat_returns_aud.ret.rolling(vol_window_l).std() * np.sqrt(12)).shift(1)\
                                 + (strat_returns_aud.ret.rolling(vol_window_mm).std() * np.sqrt(12)).shift(1)\
                                 + (strat_returns_aud.ret.rolling(vol_window_m).std() * np.sqrt(12)).shift(1)\
                                 + (strat_returns_aud.ret.rolling(vol_window_s).std() * np.sqrt(12)).shift(1)) / 4
strat_returns_aud['leverage'] = (vol_target / strat_returns_aud.rolling_vol).dropna()
# strat_returns_aud['leverage'] = vol_target / vol_df
mask = strat_returns_aud.leverage > 1
strat_returns_aud.loc[mask, 'leverage'] = 1
strat_returns_aud.loc[~strat_returns_aud.offensive_regime, 'leverage'] = 1
strat_returns_aud['ret_l'] = strat_returns_aud.ret * strat_returns_aud.leverage
strat_returns_aud['cpnl_l'] = (strat_returns_aud.ret_l + 1).cumprod()
px.bar(strat_returns_aud, y='ret_l', color='offensive_regime').show()
fig = px.line(strat_returns_aud, y='cpnl_l', color='offensive_regime', line_group='regime_group', log_y=True)#.show()
benchmark_trace = px.line(strat_returns_aud, y='spy').data[0]
benchmark_trace['line']['color'] = 'black'
benchmark_trace['line']['dash'] = 'dash'
fig.add_trace(benchmark_trace)
fig.show()
px.bar(strat_returns_aud, y='leverage', color='offensive_regime').show()
px.bar(strat_returns_aud.ret_l.rolling(12).std() * np.sqrt(12)).show()
px.bar(annual_returns(strat_returns_aud, 'ret_l'), y='ret_l').show()
cagr(strat_returns_aud.cpnl_l.dropna())
sr(strat_returns_aud.ret_l.dropna(), N=12)

In [ ]:
strat_returns

In [ ]:
strat_returns = dd_curve(strat_returns)

px.line(strat_returns, y='dd')

In [ ]:
### drawdowns numba



In [ ]:
strat_returns['dd'] = dd_curve_numba(strat_returns.cpnl.values, strat_returns.ret.values)

px.line(strat_returns, y='dd')